In [5]:
import logging
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re


import function
import imp
imp.reload(function)
import shutil
import sys

header={'content-type': 'application/json',
           'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0' }
logging.captureWarnings(True)


In [6]:
url_53 = "https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3Ad5b814ef-b2c4-4dda-a0f0-1a01886b59af%2Clang%3A%22ENG%22%7D"
url_54 ="https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3A1aacd500-ec18-4fa3-baab-3341674b13d4%2Clang%3A%22ENG%22%7D"
url_55 ="https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3Adca67a41-9d18-4bce-ab3b-442a960711a9%2Clang%3A%22ENG%22%7D"
url_56 = "https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3Ab25e03f7-e2cf-41b0-88c8-a960bab54b85%2Clang%3A%22ENG%22%7D"

In [7]:

def exceltable(url,excel_name,last_update):
    
    '''Collect the updated data
       Save data in the data pool file'''
    
    remove = ['GMID','Dataset','Last Update']
    
    # Collect data in features(inside json)
    row_table = function.table_json(url)

    # Only data of the update date bigger than last update date will be collected.
    table = function.generate(row_table,last_update)
    
    table = pd.DataFrame(table, index = list ( range( len(table) ) ) )

    if len(table)!=0:
        last_update_new = [max(table['Last Update'].tolist())]
        print(last_update_new)
        table = table.drop(remove,axis=1) 
        table.to_excel(os.getcwd()+"\\datapool"+"\\"+excel_name+'.xlsx')
        return last_update_new
    print("There are "+ str(len(table)) + " records." )
    return [last_update]

def data_collection_process():
    """create or update the data in dataset
    
        return：the newest date
    """
    step1 = input("Please input your procedure(1:Update / 2: Create) :")
    if step1 == "2":

        folder = os.getcwd()+"\\1"
        last_update = datetime(1999,1,1).date()
        if not os.path.exists(folder):
            os.makedirs(folder)

    elif step1 =="1":
        f = open("Time.txt",'r')
        date = f.readline().strip()
        print(date)
        while( len(date) == 0):
            date = input("Auto get Start Date Error, Please Input the start Date(YYYY-MM-DD): ")
            try:
                last_update = datetime.strptime(date, "%Y-%m-%d").date()
            except:
                date =""
                print("Error Input, Try again.")
        last_update = datetime.strptime(date, "%Y-%m-%d").date()   
        print("Last Update Date: ",last_update)
        if not os.path.exists(os.getcwd()+"\\dataset") or len(os.listdir(os.getcwd()+"\\dataset"))==0:
            print("Update Error, Please Create First.")
        folder = os.getcwd()+"\\datapool"
        if not os.path.exists(folder):
            os.makedirs(folder) 
        f.close()
    else :
        print("Error Input.")
        sys.exit()
    return last_update,step1

def record_last_update():
    '''save the newest date in the file (Time.txt) '''
    file = open("Time.txt",'w')
    file.write(last_update.strftime("%Y-%m-%d"))
    file.write("\n")
    file.close()
    
def add_information():
    
    '''reget Lot information and save all the data in the data_pool file'''
    
    folder = os.getcwd()+"\\datapool"
    dataset_table = os.listdir(folder)
    print(dataset_table)

    for num in range(len(dataset_table)):
        function.regetlot(folder+"\\"+dataset_table[num])

    for num in range(len(dataset_table)):
        function.tpu(folder+"\\"+dataset_table[num])

def drop_duplicate(step):
    '''drop duplicate data and save new data in the data_set file(old data in dataset will also be kept)'''
    
    folder = os.getcwd()+"\\datapool"
    dataset_table = os.listdir(folder)
    function.table_drop_dupl(folder+"\\table_53.xlsx")
    function.add_suffix(folder)

    if step == "1":
        dataset_folder = os.getcwd()+"\\dataset"
        for num in range(len(dataset_table)):
            pre_table = pd.read_excel(dataset_folder+"\\"+dataset_table[num], index_col = 0 )
            print(len(pre_table))
            aft_table = pd.read_excel(folder+"\\"+dataset_table[num], index_col = 0 )
            print(len(aft_table))
            pre_table = pd.concat([pre_table,aft_table],axis=0, ignore_index = True)
            pre_table.to_excel(dataset_folder+"\\"+dataset_table[num])
            print(len(pre_table))


In [ ]:
last_update,step = data_collection_process()

print("---------------Start to Collect Data---------------")

print("\nprocessing : Table _53 ")
last_update_53 = exceltable(url_53,'table_53',last_update)

print("\nprocessing : Table _54 ")
last_update_54 = exceltable(url_54,'table_54',last_update)

print("\nprocessing : Table _55 ")
last_update_55 = exceltable(url_55,'table_55',last_update)

print("\nprocessing : Table _56 ")
last_update_56 = exceltable(url_56,'table_56',last_update)

try:
    last_update = max(last_update_53+last_update_54+last_update_55+last_update_56)
except:
    last_update=[]
print("----------Data Collection Done.-------------")

record_last_update()

print("-------------Start Data Cleaning---------------")

print("\nProcessing: Check the number of NAN Lot and Getting TPU \n")

add_information()

print("\nProcessing: Drop Duplicate \n")

drop_duplicate(step)
print("-----------Data Cleaning Done-------------------")


        

Please input your procedure(1:Update / 2: Create) :2
---------------Start to Collect Data---------------

processing : Table _53 
https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3Ad5b814ef-b2c4-4dda-a0f0-1a01886b59af%2Clang%3A%22ENG%22%7D
100
